# Algorithm Ideas

# **ROUGH DRAFT ALL CODE IS IN PROGRESS MY WORKSTYLE IS SLOPPY I KNOW**


In [143]:
#includes
import math
import numpy as np
from matplotlib import pyplot as plt
import librosa
import sys
from scipy import signal
import scipy.io.wavfile as wav
from scipy.fft import fft, ifft
import contextlib
import os
import soundfile as sf


## Audio
### Voice recording phase: the system records voice at 5-s intervals.

### Recorded data are bandpass filtered (50 Hz~3000 Hz)

### Data are filtered with a Wiener filter. 
The Wiener filter minimizes the Mean Square Error (MSE) between the estimated random process and the desired operation. This filter is generally used to remove noise from a recorded voice.
### Short sounds and background noise are removed.
 First, an adaptive threshold to remove background noise has been used. The reference level of environmental noise must be calculated. As the noise in the disaster area is high and highly variable, an adaptive background noise reference has been defined according to the equation:
$$Ref_{noise}  = αVol_t + (1 − α)Vol_{t-1}$$
where α is the smoothing factor of Refnoise change, Volt is the average volume [dB] of current 5 s voice data, Volt−1 is the volume of previous 5 s voice data. It has been empirically found that a = 30% yields the best performance. Then, if the volume of the sound sample is lower than 1.3 times Refnoise, the algorithm identifies the sound sample as environmental noise and discards it. Sound signals that are 1.3 times higher than Refnoise are suspect sounds. Then, the algorithm checks the length of this suspect sound. As human voice sound is assumed to last more than 300 ms, sounds shorter than 300 ms are removed. After removing short sounds, this suspect sound is processed with SVM to identify possible human noise.

In [144]:
# Preprocessing

#Band-Pass

def bandpass(wavdata, sample_rate):

        low = 50
        high = 3000

        nyq = 0.5*sample_rate
        lowpass = low/nyq
        highpass = high/nyq
        b, a = signal.butter(3, [lowpass, highpass], btype="band")
        filtered_wav = signal.filtfilt(b, a, wavdata, padtype=None)

        # wav.write(str(wavname+"_filtered.wav"), sample_rate, filtered_wav.astype("float32"))
        return filtered_wav




In [145]:
def wiener_filter(wavdata):
    return signal.wiener(wavdata)

In [146]:
def short_sound_removal(wavdata, pwa_volume, smoothing_factor, sample_rate):
    # pwa_volume -- average volume of the previous 5sec clip.
    # smoothing factor -- float between 0 and 1, 0.3 is empirically tested to be best
    volume = sum(wavdata)/len(wavdata)
    refnoise = smoothing_factor*volume - (1-smoothing_factor)*pwa_volume
    # Then scan for segments greater than 300ms of nonzeros
    final_sounds = get_long_sounds(wavdata, refnoise, 0.3, sample_rate)
    return final_sounds


def get_long_sounds(samples, threshold, min_duration, sample_rate):
    # Find all non-ambient samples
    non_ambient = np.where(samples > threshold, samples, 0)
    longer_than = min_duration*sample_rate
    found = False
    start = 0
    long_sounds = list()
    for i in range(len(non_ambient)):
        #New start
        if (non_ambient[i] > 0) and (not found):
            start = i
            found = True
        #Have started, found a 0 after threshold
        elif (non_ambient[i]<0) and (i-start >= longer_than):
            found = False
            #record the sound
            long_sounds.append(np.array(non_ambient[start:i-1]))
        #Have started, found a 0 before threshold
        elif (non_ambient[i]<0) and (i-start < longer_than):
            found = False
    
    return long_sounds



### Segmentation.
 The 5-s audio signal, after removing short sounds and background noise, is broken into shorter audio samples of 10 ms.
### Audio statistical features are computed for these shorter 10-ms audio samples.
### SVM Classification.
 Sounds are differentiated in human voice or noise.

In [147]:
def splitter(wavdata, sample_rate):
    # read
    # Define the clip length in seconds
    clip_length = 0.01

    # Calculate the number of clips
    num_clips = int(np.ceil(len(wavdata) / (sample_rate * clip_length)))

    # Create an empty list to store the clips
    clips = []

    # Loop over the clips
    for i in range(num_clips):
        # Calculate the start and end samples of the clip
        start_sample = int(i * sample_rate * clip_length)
        end_sample = min(int((i + 1) * sample_rate * clip_length), len(wavdata))

        # Extract the clip from the data
        clip = wavdata[start_sample:end_sample]

        # Append the clip to the list
        clips.append(clip)
    return clips


In [148]:
# Test splitter


### Energy Entropy

#### $$H(X) = −\Sigma^{n}_{i=1}p(x_i)\log{p(x_i)}$$

where X is a discrete RV with pdf $p(x)$ and readings $\{x_i\}_{i=1}^n$

In [149]:
def compute_entropy(hist): 
    # Calculate the entropy of the histogram
    return -np.sum(hist * np.log2(hist + 1e-6)) #Safeguard value to prevent log of 0

### Signal Energy

#### $$E_s = \int^{\infty}_{-\infty}|x(t)|^2dt$$

Where $x(t)$ is a continuous-time signal



In [150]:
## May need to look into this one more -- Not sure if applying correctly
## Basically square all values, sum them up, divide by amount of values for a discrete integral instead of continuous
def compute_signal_energy(histogram):
    signal_energy = np.sum(np.square(histogram))/len(histogram)
    return signal_energy

### Zero Crossing Rate

#### $$ZCR = \frac{1}{T-1}\Sigma^{T-1}_{t=1}1_{R>0}(s_ts_{t-1})$$

where $s$ is a voice signal of length $T$ and $1_{R>0}$ is an indicator function


In [151]:
def compute_zero_cross(clip):
    # Set indicator function values of main hist
    signage = np.sign(clip)
    #Find sum
    total = np.sum(np.abs(np.diff(signage)))
    return total/((len(clip)-1))


### Spectral Roll-Off

#### $$\Sigma^{R_t}_{n=1}M_t[n] = 0.85 \times \Sigma^{N}_{n=1}M_t[n]$$

Where $M_t[n]$ is the magnitude of the Fourier Transform at frame $t$ and frequnecy bin $n$ and $R_t$ is the frequency

In [152]:
def compute_spectral_rolloff(fourier, sample_rate):
    import librosa

    # calculate the power spectrum
    magnitude = np.abs(fourier) ** 2

    #accumulate
    cumulative_sum = np.cumsum(magnitude, axis=0)

    # find the frequency bin index at which 85% of the power is contained
    total_power = np.sum(magnitude)
    cutoff_index = np.argmax(cumulative_sum > 0.85 * total_power)

    # calculate the spectral rolloff frequency as the frequency bin center
    # corresponding to the cutoff index
    spectral_rolloff = librosa.core.fft_frequencies(sr=sample_rate, n_fft=fourier.shape[0])[cutoff_index]

    # print the spectral rolloff frequency in Hz
    return spectral_rolloff


### Spectral Centroid

#### $$C = \frac{\Sigma^{N-1}_{n=0}f(n)x(n)}{\Sigma^{N-1}_{n=0}x(n)}$$

where $x(n)$ represents the weighted frequency value, or magnitude, of bin $n$, and $f(n)$ represents the center frequency of that bin

In [153]:
def compute_spectral_centroid(clip, sample_rate):
    magnitudes = np.abs(np.fft.rfft(clip))  # Compute the magnitude spectrum
    freqs = np.fft.rfftfreq(len(clip), d=1/sample_rate)  # Compute the frequency bins
    return np.sum(magnitudes * freqs) / np.sum(magnitudes)  # Compute the weighted average of frequency bins

### Spectral Flux

#### $$F_t = \Sigma^{N}_{n=1}(N_t[n]-N_{t-1}[n])^2$$

where $N_t[n]$ and $N_{t-1}[n]$ are the normalized Fourier transform at frames t and t-1

In [154]:
def compute_spectral_flux(fourier_curr, fourier_prev):
    # compute magnitude spectrum for each clip
    magnitude1 = np.abs(fourier_curr)
    magnitude2 = np.abs(fourier_prev)

    # compute spectral flux between the two clips
    spectral_flux = np.sum(np.abs(magnitude2 - magnitude1))

    # print the spectral flux value
    return spectral_flux
    

### Then, a driver that gathers all the information for the clips

For each 5s clip, divide into 1000 5ms clips.
Need to discuss whether we are evaluating 1 5s clip or 1000 5ms clips for analysis.  I vote 1 5s clip with 6 attributes

In [155]:
def collect_data(wavname, ref_noise):
    sample_rate, wavdata = wav.read(wavname)
    #filter the audio bandpass
    bandpass_wav = bandpass(wavdata, sample_rate)
    #Wiener filter
    wiener = wiener_filter(bandpass_wav)
    #Remove short sounds
    candidates = short_sound_removal(wiener, ref_noise, 0.3, sample_rate)
    
    features = list() #stores for each clip
    #Preprocessing of original audio complete, begin splitting and feature definition.
    for candidate in candidates:
        #split into 10ms sections
        splits = splitter(candidate, sample_rate)
        #take fourier of first clip for flux purposes
        prev_fourier = librosa.stft(splits[0])
        min_prev = min(prev_fourier)
        max_prev = max(prev_fourier)
        prev_fourier = [(x-min_prev/(max_prev-min_prev)) for x in prev_fourier]
        #start at second clip for flux purposes
        for clip in splits[1:]:
            fourier = librosa.stft(clip)
            min_fourier = min(fourier)
            max_fourier = max(fourier)
            normalized_fourier = [(x-min_fourier/(max_fourier-min_fourier)) for x in fourier]
            # Calculate the decibel levels for the clip
            db = librosa.amplitude_to_db(np.abs(clip), ref=np.max)
            # Calculate the histogram of the decibel levels
            hist, bins = np.histogram(db, bins='auto', density=True)
            features.append(np.array([compute_entropy(hist), compute_signal_energy(hist), compute_zero_cross(hist), compute_spectral_rolloff(fourier, sample_rate), compute_spectral_centroid(fourier, sample_rate), compute_spectral_flux(normalized_fourier, prev_fourier)]))

            #save for next clips flux
            prev_fourier = normalized_fourier

    # For entropy, ZCT, Roll Off, Centroid, compute SD
    std_entropy = np.std([clip_features[0] for clip_features in features])
    std_ZCT = np.std([clip_features[2] for clip_features in features])
    std_rolloff = np.std([clip_features[3] for clip_features in features])
    std_centroid = np.std([clip_features[4] for clip_features in features])
    # For Signal Energy, Flux, compute SD by Squared Mean
    # Not sure about the difference between these two, ask Prof Murphy maybe? For now, just use regular std
    std_sig_nrg = np.std([clip_features[1] for clip_features in features])
    std_flux = np.std([clip_features[5] for clip_features in features])
    #Features prepped for SVM
    return (std_entropy, std_sig_nrg, std_ZCT, std_rolloff, std_centroid, std_flux)
        


In [156]:
#Testdrive
print(collect_data("input1_mono.wav"))

C:\Users\terra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=480
  warnings.warn(
C:\Users\terra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\fft\_pocketfft.py:70: ComplexWarning: Casting complex values to real discards the imaginary part
  r = pfi.execute(a, is_real, is_forward, fct)
C:\Users\terra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=161
  warnings.warn(
C:\Users\terra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too lar

(0.20082477178883112, 0.0004529460258841722, 0.09173359953046255, 499.085310037785, 2.9381627519454868e-09, 870468.7725904444)


# SVM Classification
Going to have to train an SVM somehow.  I think the best way to do it would be to record a long wav on our current mic setup, split it, mark which sections had a voice and which ones did not (i.e. set it up in the living room for a bit).  Then listen and manually track voices.  After that, use as training data.  If we are measuring 5 second clips, we are going to want perhaps 30 minutes of volume input.  Can also look for an SVM model online that is built or some voice data, I'm sure it exists out there.  But the model might work better if it was built on the audio from our mic.


In [157]:
# Build SVM Model

# For REFNOISE -- Take ambient, compute average, use that for threshold of previous 5 second clip.

In [158]:
# Run snippet through SVM Model

## $CO_2$

## LIDAR

## Thermal Camera (?)

### Helpful Links
